## This notebook loads EPH data, cleans it and arranges it to be used as 'training sets'. 
That is, for fitting any Machine Learning model.

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

import numpy as np
pd.options.display.max_columns = 99

from IPython.display import display, HTML

import glob

In [ ]:
startyr = 2020
endyr = 2021

In [2]:
radio_ref = pd.read_csv('./../data/info/radio_ref.csv')
# radio_ref[['PROV','NOMPROV','DPTO', 'NOMDPTO']].drop_duplicates().to_csv('./../data/DPTO_PROV.csv', index = False)
dpto_region = pd.read_csv('./../data/info/DPTO_PROV_Region.csv')
radio_ref = radio_ref.merge(dpto_region)
AGLO_Region = radio_ref[['AGLOMERADO', 'Region']].drop_duplicates()

# Decision sobre cual es la region de un aglomerado. GBA tiene que ir a Gran Buenos Aires, aunque algunos de sus radios en partidos como Rodriguez, Escobar, etc sean region pampeana.
# Viedma Patagones, se tendria que tirar de un lado, y la mayoria de sus radios, son Patagonia.
# Se tiene que corregir a mano, porque el AGLO 0 SI tiene varias regiones.

AGLO_Region = AGLO_Region.loc[~((AGLO_Region.AGLOMERADO == 33) & (AGLO_Region.Region == 'Pampeana'))]
AGLO_Region = AGLO_Region.loc[~((AGLO_Region.AGLOMERADO == 93) & (AGLO_Region.Region == 'Pampeana'))]

### Match column names

names_censo = ['IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO',
    'V01', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14', 'H13',
      'P07', 'P08', 'P09', 'P10', 'P05']


names_EPH = ['IX_TOT','CH04','CH06','CONDACT', 'AGLOMERADO',
    'IV1', 'IV3', 'IV4','IV5','IV6','IV7','IV8','IV10','IV11','II1','II2','II7','II8','II9',
    'CH09','CH10','CH12','CH13','CH15']

col_mon = [u'P21', u'P47T', u'PP08D1', u'TOT_P12', u'T_VI', u'V12_M', u'V2_M', u'V3_M', u'V5_M']

cpi = pd.read_csv('./../data/info/indice_precios.csv', index_col=0)
cpi.index = pd.date_range("1943-01", periods=len(cpi), freq = 'M')
cpi = cpi['2003':]


In [3]:
## Tasa de inflacion de los ultimos 12 meses. Se usa para completar la inflacion del ultimo mes antes que se mida.
r = cpi.pct_change().tail(12).mean()[0]

## Estirar la serie de CPI hasta el dia de hoy (para precios actualizados)
cpi = pd.concat([cpi, pd.DataFrame([], index = pd.date_range(cpi.index[-1] + 1, pd.datetime.today() + pd.DateOffset(months=1), freq = 'M'))])


last_valid_ix = cpi.dropna().iloc[-1] # Level of the cpi index in last valid month

# np.ones(len(cpi))
cpi['avg_rate'] = pd.Series((1 + r)**range(len(cpi)), index = cpi.index)  ## Compute the exponential curve

# Bring to actual level
ratio = cpi.loc[last_valid_ix.name].avg_rate / cpi.loc[last_valid_ix.name]['index'] 
cpi['avg_rate'] = cpi['avg_rate']/ratio  

# Complete cpi series up to present and remove exponential
cpi['index'] = cpi['index'].fillna(cpi['avg_rate'])
cpi = cpi.drop('avg_rate', axis = 1)

In [4]:
nivel_precios_actual = cpi.tail(1)['index']
display(nivel_precios_actual)

cpi = cpi.groupby(pd.TimeGrouper(freq='Q')).mean().loc[str(startyr):str(endyr)][['index']]#.to_csv(...)
cpi.set_index('index')
cpi.index.name = 'Q'
#
indice_precios = cpi

from pandas.tseries.offsets import MonthEnd

## Cargar EPHs

In [ ]:

path ='./../../../../../../home/miglesia/Documents/EPH/microdatos/' # use your path


for y in range(startyr, endyr):
    yr = str(y)[2:]
    allFiles = glob.glob(path + 'hogar/*'+str(yr)+'.txt')
    frame = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        df = pd.read_csv(file_,index_col=None, header=0, delimiter = ';',
                        usecols = ['CODUSU','ANO4','TRIMESTRE','IX_TOT', 'AGLOMERADO',
        'IV1', 'IV3', 'IV4','IV5','IV6','IV7','IV8','IV10','IV11','II1','II2','II7','II8','II9']) 
        ['II2', 'IV5', 'IX_TOT', 'II7', 'IV4', 'II1', 'IV7', 'IV6', 'IV11', 'IV8', 'IV3', 'II8', 'IV1', 'IV10', 'II9']
        
        print(len(df))
        list_ += [df]
    df = pd.concat(list_)

    # Correcciones Respuestas. Para que matchee censo
    df = df.loc[df.IV1 != 9]
    df['IV10'] = df['IV10'].map({1: 1, 2: 2, 3: 2, 0: 0, 9: 9})
    df['II9'] = df['II9'].map({1: 1, 2: 2, 3: 2, 4: 4, 0: 0})
    df['II7'] = df['II7'].map({1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 6, 8: 6, 9: 6, 0: 0})
    df['II9'] = df['II9'].map({1: 1, 2: 2, 3: 2, 4: 4, 0: 0})
    df['IX_TOT'] = df['IX_TOT'].clip(0, 8)
    
    hogar = df
    hogar = hogar.drop_duplicates()
    print(hogar.shape)

    allFiles = glob.glob(path + 'individual/usu_individual*'+str(yr)+'.txt')
    frame = pd.DataFrame()
    list_ = []
    for file_ in allFiles:
        print(file_)
    #     print(file_)
        df = pd.read_csv(file_,index_col=None, header=0, delimiter = ';',
                         usecols = ['CODUSU','ANO4','TRIMESTRE','CH04','CH06', 'AGLOMERADO', 'CH09','CH10','CH12','CH13','CH15'] +\
                         ['CH07', 'ESTADO','CAT_INAC','CAT_OCUP','PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59', 'PP07H', 'PP07I', 'PP07J', 'PP07K',
                         'P47T', 'V3_M', 'T_VI', 'V12_M', 'TOT_P12', 'V5_M','V2_M', 'PP08D1', 'P21'])
        df = df.rename(columns = {'ESTADO': 'CONDACT'})

#         display(df.head())
# revisar estado, condact, cat ocup, cat inac.
    # For the regression training set. But for these the ANO4 TRIMESTRE is important.. Also we need more memory.
    #                      ['P21','P47T',,'CH08','CH16','TOT_P12','T_VI','V10_M','V11_M','V12_M','V18_M','V19_AM','V21_M','V2_M','V3_M',
    #             'V4_M','V5_M','V8_M','V9_M','PP08D1','PP08D4','PP08F1','PP08F2','PP08J1','PP08J2','PP08J3','PP10A','PP10C','PP10D','PP10E'])
#         print(len(df))
        list_ += [df]
    df = pd.concat(list_)

    # Correcciones Respuestas. Para que matchee censo
    df['CH15'] = df['CH15'].map({1:1, 2:1, 3:1, 4:2, 5:2, 9:0})
    df['CH06'] = df['CH06'].clip(0)
    df['CH09'] = df['CH09'].map({1:1, 2:2, 0:2, 3:2})
    df.loc[df['CH06'] < 14, 'CONDACT'] = 0 # Menores de 14 van con CONDACT 0, como en el Censo
    
    ## En Censo, Jardin y educacion especial no preguntan terminado si/no.
    df['CH12'] = df.CH12.replace(99, 0)
    df.loc[df.CH12.isin([0, 1, 9]), 'CH13'] = 0

#     df['MAYOR'] = df['CH06'] >= 14 
#     df['MAYOR'] = df['CH06'] // 7
#     df['CONDACT'] = df['CAT_OCUP'].fillna(-1)

    indiv = df
    indiv = indiv.dropna(subset = ['P47T'])
    print(indiv.shape)

    indiv_table = indiv[list(indiv.columns.difference(hogar.columns)) + ['CODUSU', 'ANO4', 'TRIMESTRE', 'AGLOMERADO']]
    EPH = hogar.merge(indiv_table, on = ['CODUSU', 'ANO4', 'TRIMESTRE', 'AGLOMERADO'], indicator = True)

    print('Hogar - Indiv merged:')
    print(EPH.shape)

    
#     EPH = EPH.loc[EPH.P47T != -9]
    
    EPH = EPH.merge(AGLO_Region)

    EPH_no_aglo = EPH.copy(); 
    EPH_no_aglo['AGLOMERADO'] = 0

    EPH = pd.concat([EPH, EPH_no_aglo]).reset_index(drop = True)

    print('No aglo agregado:')
    print(EPH.shape)
    
    # Quarters / deflation
    EPH['Q'] = EPH.ANO4.astype(str) + ':' + (3*EPH.TRIMESTRE).astype(str)
    EPH['Q'] = pd.to_datetime(EPH['Q'], format='%Y:%m') + MonthEnd(1)
#     cpi_ultimo_Q = indice_precios['index'].values[-1]
    cpi_mes_actual = nivel_precios_actual.values[0]
    
    EPH[col_mon] = cpi_mes_actual*EPH[col_mon].div(EPH[['Q'] + col_mon].merge(indice_precios, on = 'Q', how = 'left')['index'].values, 0)
    
    # 2018Q3 -> Mar19 1.3156
    # 2018Q3 -> Abr19 1.361
#     EPH[col_mon] = 1.361*EPH[col_mon]
    
    EPH[col_mon] = EPH[col_mon].round()
    
    print('deflactado:')
    print(EPH.shape)
    display(EPH[col_mon].mean())
    
    
    training = EPH.rename(columns = dict(zip(names_EPH, names_censo)))
    training.to_csv('./../data/training/EPHARG_train_'+str(yr)+'.csv', index = False)

### Ranking de AGLOS y Regiones

In [ ]:
df_list = []
for yr in [str(s) for s in [2006, 2011, 2016]]:
# for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    train = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')
    train = train.loc[train.P47T >= -0.001].fillna(0)#.sample(400000)
    df_list += [train]
    
train_df = pd.concat(df_list)

AGLO_rk = train_df.loc[train_df.CAT_OCUP == 3].groupby(['AGLOMERADO'])[['P47T']].mean().sort_values('P47T').reset_index().reset_index().rename(columns = {'index':'AGLO_rk'})
Reg_rk = train_df.loc[train_df.CAT_OCUP == 3].groupby(['Region'])[['P47T']].mean().sort_values('P47T').reset_index().reset_index().rename(columns = {'index':'Reg_rk'})

AGLO_rk['AGLO_rk'] = AGLO_rk.AGLO_rk/AGLO_rk.AGLO_rk.max()
AGLO_rk.to_csv('./../data/AGLO_rk', index = False)
Reg_rk['Reg_rk'] = Reg_rk.Reg_rk/Reg_rk.Reg_rk.max()
Reg_rk.to_csv('./../data/Reg_rk', index = False)

# check it out
# AGLO_rk.merge(pd.read_csv('./../data/info/aglo_labels.csv'))

## Guardar training dataset

In [ ]:
AGLO_rk = pd.read_csv('./../data/info/AGLO_rk')
Reg_rk = pd.read_csv('./../data/info/Reg_rk')

df_list = []
for yr in [str(s) for s in range(startyr, endyr)]:
# for yr in [str(s) for s in range(startyr, endyr)]:
    print(yr)
    train = pd.read_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv')#.drop(['AGLO_rk', 'Reg_rk'], axis = 1)
    train = train.loc[train.P47T >= -0.001].fillna(0)
    train = train.merge(AGLO_rk[['AGLOMERADO', 'AGLO_rk']]).merge(Reg_rk[['Region', 'Reg_rk']])
    train.to_csv('./../data/training/EPHARG_train_'+yr[2:]+'.csv', index = False)

## Listo. Salvado el training set.

# Older code

### Filtrando censo a los DPTO elegidos

In [18]:
# import dask.dataframe as dd
# from dask.diagnostics import ProgressBar

# #Esto es para extraer las viviendas, hogares y personas de los partidos (DPTOs) en cuestion.

# VIVIENDA = dd.read_csv('./VIVIENDA.csv', sep = ';', usecols = ['VIVIENDA_REF_ID', 'RADIO_REF_ID', 'TIPVV', 'V01'])
# VIVIENDA = VIVIENDA.merge(radio_ref[['RADIO_REF_ID', 'DPTO']])
# VIVIENDA_ = VIVIENDA.loc[VIVIENDA.DPTO.isin(seleccion_DPTOS)]
# with ProgressBar():
#     VIVIENDA_REF_ID_sel = VIVIENDA_['VIVIENDA_REF_ID'].values.compute()

# HOGAR = dd.read_csv('./HOGAR.csv', sep = ';', usecols = ['HOGAR_REF_ID', 'VIVIENDA_REF_ID', 'H05', 'H06', 'H07', 'H08',
#        'H09', 'H10', 'H11', 'H12', 'H13', 'H14', 'H15', 'H16', 'PROP', 'TOTPERS']) # csv is too big, so it is dask-loaded. Not sure it's efficient thou
# # For example computing len takes ages
# # len(HOGAR.VIVIENDA_REF_ID)
# HOGAR_ = HOGAR.loc[HOGAR.VIVIENDA_REF_ID.isin(VIVIENDA_REF_ID_sel)]
# with ProgressBar():
#     HOGAR_REF_ID_sel = HOGAR_['HOGAR_REF_ID'].values.compute()

# PERSONA = dd.read_csv('./PERSONA.csv', sep = ';', usecols = ['PERSONA_REF_ID', 'HOGAR_REF_ID', 'P01', 'P02', 'P03', 'P05', 'P06',
#        'P07', 'P12', 'P08', 'P09', 'P10', 'CONDACT'])
# PERSONA_ = PERSONA.loc[PERSONA.HOGAR_REF_ID.isin(HOGAR_REF_ID_sel)]

# tabla_censo = VIVIENDA_.merge(HOGAR_)#.merge(PERSONA_)

# IX_TOT = tabla_censo.groupby('HOGAR_REF_ID').count().iloc[:, 0].reset_index()
# IX_TOT.columns = ['HOGAR_REF_ID', 'IX_TOT']

# tabla_censo = tabla_censo.merge(IX_TOT)

# with ProgressBar():
#     table = tabla_censo.compute()


[########################################] | 100% Completed | 10.0s
[########################################] | 100% Completed | 27.6s
[########################################] | 100% Completed | 44.5s


### Making EPH and Censo answers uniform

In [19]:
# Approach: modify Census to fit EPH
# Armonizar para adecuar dataset Censo a las opciones rta de EPH. No correr 2 veces
table['V01'] = table['V01'].map({1:1, 2:6, 3:6, 4:2, 5:3, 6:4, 7:5, 8:6})
table['H06'] = table['H06'].map({1:1, 2:2, 3:3, 4:4, 5:5, 6:6, 7:7, 8:9})
table['H09'] = table['H09'].map({1:1, 2:2, 3:3, 4:4, 5:4, 6:4})
table['H16'] = table['H16'].clip(0, 9)
table['H14'] = table['H14'].map({1:1, 2:4, 3:2, 4:2, 5:4, 6:3, 7:4, 8:9})
table['H13'] = table['H13'].map({1:1, 2:2, 4:0})
# table['P07'] = table['P07'].map({1:1, 2:2, 0:2})

# saber de que aglo es la persona. Se usa los resultados de cada aglo.
table = table.merge(radio_ref[['RADIO_REF_ID','AGLOMERADO']]) 

# table = table.set_index('PERSONA_REF_ID')

In [20]:
# Only once to save time in the future
# name =  'bolivar'#''
# name =  'vlopez_rodriguez'#''
name =  'rand'#''
table.to_csv('./sample_censo_table'+str(n).zfill(3)+name+'.csv', index = False)

# table = pd.read_csv('./sample_censo_table'+str(n).zfill(3)+'.csv')#.csv


In [21]:
with ProgressBar():
    PERSONA_ = PERSONA_.compute()

table = table.merge(PERSONA_)

table['P07'] = table['P07'].map({1:1, 2:2, 0:2})

# Only once to save time in the future
table.to_csv('./sample_censo_table'+str(n).zfill(3)+name+'.csv', index = False)


[########################################] | 100% Completed |  1min  2.4s


In [74]:
# AGLOS_censo = 
radio_ref_sel = radio_ref.loc[table['RADIO_REF_ID'].drop_duplicates().values]
aglos_sel = radio_ref_sel.AGLOMERADO.unique()

## Entradas de censo de los DPTOs elegidos

print(table.shape) #cuanta (gente, variables)?
table.sample(5)

(34190, 34)


,VIVIENDA_REF_ID,RADIO_REF_ID,TIPVV,V01,DPTO,HOGAR_REF_ID,H05,H06,H07,H08,...,P02,P03,P05,P06,P07,P12,P08,P09,P10,CONDACT
142,4681791,13813,1,1.0,6105,4307053,1,4.0,1,1,...,1,57,1,0,1,2,2,2,1,1
19158,4691129,13853,2,NaN,6105,4314259,0,NaN,0,0,...,1,48,2,221,1,0,2,2,1,0
5391,4684094,13819,1,1.0,6105,4308935,1,4.0,1,1,...,1,47,1,0,1,2,2,2,1,1
22580,4692731,13861,1,1.0,6105,4315487,1,4.0,1,1,...,1,60,1,0,1,2,2,2,1,1
9618,4685920,13824,1,1.0,6105,4310454,1,1.0,1,1,...,1,62,1,0,1,2,2,4,2,1


## Cargar EPH

### Adoptar mismos nombres de columnas

In [89]:
# Misma info, distinto nombre. 
# Censo INDEC 
md_1 = table[['IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', #las que no se erran, cant pers, sexo, edad, act, aglo
    'V01', 'H05', 'H06', 'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14', 'H13',
      'P07', 'P08', 'P09', 'P10', 'P05']] #las x que buscan matches un poquito mas laxamente


#Mismas cosas, distinto nombre de columna para
# EPH INDEC
md_2 = EPH[['IX_TOT','CH04','CH06','CONDACT', 'AGLOMERADO',
    'IV1', 'IV3', 'IV4','IV5','IV6','IV7','IV8','IV10','IV11','II1','II2','II7','II8','II9',
    'CH09','CH10','CH12','CH13','CH15']]

# # Now we want to see in each column what are the percentages, as a clue to where there can be issues
# # OK control check. Control there is less likely confusion. 

# for i in range(len(md_1.columns))[:2]: 
#     print('\n')
#     for md in [md_1, md_2]:
#         col = md.columns[i]
#         print(col)
#         print(md[col].value_counts().sort_index()/len(md))

md_2.columns = md_1.columns

In [90]:
import numpy as np


In [91]:
### The 'y' variables will be predicted. K nearest neighbors is used.
## Variables in the EPH survey but not in the Censo.
## Preguntas de EncuestaPH que no estan en Censo.

y_cols = ['P21','P47T','CAT_INAC','CAT_OCUP','CH07','CH08','CH16','TOT_P12','T_VI','V10_M','V11_M','V12_M','V18_M','V19_AM','V21_M','V2_M','V3_M','V4_M','V5_M','V8_M','V9_M',
         'PP07G1', 'PP07G2', 'PP07G3', 'PP07G4', 'PP07G_59','PP07H','PP07I','PP07J','PP07K','PP08D1','PP08D4','PP08F1','PP08F2','PP08J1','PP08J2','PP08J3','PP10A','PP10C','PP10D','PP10E']

#Remove ill predicted ones after trying them out
y_cols = list(set(y_cols) - set(['PP10A', 'V11_M', 'PP08D4', 'PP08J3', 'PP08F1', 'V18_M', 'V10_M',
       'V8_M', 'V4_M', 'PP08F2', 'V21_M', 'V9_M', 'PP08J2', 'PP08J1',
       'V19_AM']))



train = md_2.join(EPH[y_cols]).fillna(0)
test = md_1.fillna(0).astype(int)


In [92]:
md_1.columns

Index(['IX_TOT', 'P02', 'P03', 'CONDACT', 'AGLOMERADO', 'V01', 'H05', 'H06',
       'H07', 'H08', 'H09', 'H10', 'H11', 'H12', 'H16', 'H15', 'PROP', 'H14',
       'H13', 'P07', 'P08', 'P09', 'P10', 'P05'],
      dtype='object')

In [93]:
train.to_csv('./train.csv', index = False)

In [94]:
len(train)

58977

In [95]:
# Se entrena en .8% de la poblacion... con solo tener La Matanza y La Plata. Parece poco: error prone.
print(train.shape)
print(test.shape)
len(train)/len(test)

(58977, 50)
(34190, 24)


1.7249780637613337

In [18]:
# Las variables principales que no se erran. Personas en hogar, su sexo, su decil de edad (~ grupo 6 anios) y condicion de actividad, o sea si sale a trabajar o 'esta en casa'.

# train[['IX_TOT', 'P02', 'P03', 'CONDACT']].nunique().values
# train[['IX_TOT', 'P02', 'P03', 'CONDACT']].drop_duplicates()

In [98]:
len(test[x_cols].drop_duplicates())/len(test[x_cols])
# test[x_cols]

0.6068441064638783

In [97]:
# The columns that we will be making predictions with.
x_cols = md_1.columns
from sklearn.neighbors import KNeighborsRegressor
# Create the knn model.
# Look at the five closest neighbors.
knn = KNeighborsRegressor(n_neighbors=1)

In [20]:
# Fit the model on the training data.
train = train.sample(frac = 1)
#Add noise to reduce tie situations in a more or less random way
knn.fit(train[x_cols] + .1*(np.random.random_sample(train[x_cols].shape)-0.5), train[y_cols])

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=1, p=2,
          weights='uniform')

In [21]:
# %%timeit

# Make point predictions on the test set using the fit model.
predictions = knn.predict(test[x_cols])

In [22]:
pred_df = pd.DataFrame(predictions, columns=y_cols, index = test.index)

result = test.join(pred_df)

In [23]:
pd.options.display.max_columns = 990
pd.options.display.max_rows = 999

print(result.shape)
result.head()

(1763261, 50)


,IX_TOT,P02,P03,CONDACT,AGLOMERADO,V01,H05,H06,H07,H08,H09,H10,H11,H12,H16,H15,PROP,H14,H13,P07,P08,P09,P10,P05,V12_M,PP08D1,PP07K,PP07G4,V2_M,CAT_OCUP,V3_M,TOT_P12,PP07G2,PP10C,T_VI,CH16,CH07,PP07H,PP10E,PP07G3,PP07J,PP07G_59,PP07I,PP10D,CH08,PP07G1,CAT_INAC,P47T,P21,V5_M
0,3,1,8,100,0,1,1,4,1,1,2,1,1,2,3,2,1,2,1,1,2,4,2,1,0.0,30000.0,1.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,30000.0,30000.0,0.0
1,3,2,7,300,0,1,1,4,1,1,2,1,1,2,3,2,1,2,1,1,2,7,2,1,0.0,0.0,0.0,0.0,7200.0,0.0,0.0,0.0,0.0,0.0,7345.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,7345.0,0.0,145.0
2,3,1,2,300,0,1,1,4,1,1,2,1,1,2,3,2,1,2,1,1,1,4,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,3.0,0.0,0.0,0.0
3,4,1,8,100,0,1,1,4,1,1,2,1,1,2,5,3,4,2,1,1,2,2,1,1,0.0,-9.0,1.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,0.0,-9.0,-9.0,0.0
4,4,1,3,100,0,1,1,4,1,1,2,1,1,2,5,3,4,2,1,1,2,5,1,1,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,4000.0,4000.0,0.0


In [24]:
result.to_csv('./test_result_'+str(n).zfill(3)+'dpto_1.csv')
# result.to_csv('./test_result_'+str(n).zfill(3)+'dpto.csv')

# Load result

In [25]:
res_1 = pd.read_csv('./test_result_'+str(n).zfill(3)+'dpto_1.csv')

In [26]:
# res_0 = pd.read_csv('./test_result_'+str(n).zfill(3)+'dpto.csv')

In [27]:
# porcentaje error por reshuffle: 
perr_ = np.round(100*abs(res_0.sum() - res_1.sum())/res_0.sum(), 1).sort_values()

# perr_.tail(15)#.index
# Bad ones: ['PP10A', 'V11_M', 'PP08D4', 'PP08J3', 'PP08F1', 'V18_M', 'V10_M',
#        'V8_M', 'V4_M', 'PP08F2', 'V21_M', 'V9_M', 'PP08J2', 'PP08J1',
#        'V19_AM']


NameError: name 'res_0' is not defined

In [ ]:
import numpy as np
# Agregar $$$. En millones de usd
# En millones de usd (USD = 30 ARS)
_USD = 30.5 #ARS
np.round(res_1.sum()/_USD/1e6, 1).sort_values().tail(14)

#PPALES
# negocio que no trabajo no laborable (V9_M)
# alquiler no laborable (V8_M)
# indemnizacion despido no laborable (V3_M)
# comision Ocupacion ppal (PP08F1)
# cuota alimentos no laborable (V12_M)
# subsidio ayuda social no laborable (V5_M)
# TOTAL otras ocupacions(TOT_P12)
# jubilacion no laborable (V2_M)
# TOTAL no laborables (T_VI)
# sueldo Ocupacion ppal(PP08D1)
# TOTAL Ocupacion ppal (P21)
# TOTAL TOTAL (P47T)

In [ ]:
# np.round(res_1.sum()/1e6/_USD, 1).sort_values().tail(14).index

In [ ]:
PERS_DPTO = table[['PERSONA_REF_ID', 'RADIO_REF_ID']].merge(radio_ref_sel[['RADIO_REF_ID', 'DPTO' #, 'NOMDPTO', 'NOMPROV'
                                                                      ]]).drop(['RADIO_REF_ID'], axis = 1)

res = res_1
res_DPTO = res.merge(PERS_DPTO, on = 'PERSONA_REF_ID')

#en miles de USD
res_byDPTO = res_DPTO.groupby(['DPTO'])[[  'V3_M', 'V12_M', 'V5_M', 'TOT_P12',
       'V2_M', 'T_VI', 'PP08D1', 'P21', 'P47T']].sum()

np.round(100*res_byDPTO.div(res_byDPTO.P47T, axis = 0), 1).sort_values(by = 'P21').head() #percentage
# np.round(res_byDPTO/1e3/_USD, 1) #in 1000 USD

In [ ]:
PERS_DPTO = table[['PERSONA_REF_ID', 'RADIO_REF_ID']].merge(radio_ref_sel[['RADIO_REF_ID', 'DPTO', 'NOMDPTO', 'NOMPROV']]
                                                           )#.drop(['RADIO_REF_ID'], axis = 1)

res = res_1
res_DPTO = res.merge(PERS_DPTO, on = 'PERSONA_REF_ID')

# variables = ['V9_M', 'V8_M', 'PP08F1', 'V3_M', 'V12_M', 'V5_M', 'TOT_P12',
#        'V2_M', 'T_VI', 'PP08D1', 'P21', 'P47T']
variables = [  'V3_M', 'V12_M', 'V5_M', 'TOT_P12',
       'V2_M', 'T_VI', 'PP08D1', 'P21', 'P47T']
#en miles de USD
# res_byDPTO = res_DPTO.groupby(['DPTO', 'NOMDPTO', 'NOMPROV'])[variables].sum()
res_byDPTO = res_DPTO.groupby(['RADIO_REF_ID'])[variables].sum()

# np.round(100*res_byDPTO.div(res_byDPTO.P47T, axis = 0), 1).sort_values(by = 'P21').head() #percentage
np.round(res_byDPTO/1e3/_USD, 1) #in 1000 USD

In [ ]:
# Save info at 'radio' level
res_byDPTO.to_csv('res_byradio_sample_'+str(n).zfill(3)+'.csv')

In [ ]:
# Desoc, NA = 0. Not good.
variables = ['PP07J', #turno habitual
 'PP10D', #Desoc. Ha trabajado alguna vez?
 'PP10C', #Desoc. Hizo changa mientras buscaba?
 'PP07K', # Oc. ppal. Inc. serv. dom. Cobra con recibo
 'PP07G2', # Oc. ppal. Inc. serv. dom. aguinaldo
 'PP07G_59', # Oc. ppal. Inc. serv. dom. ninguno
 'PP07G3', # Oc. ppal. Inc. serv. dom. dias enfermedad
 'PP10E', # Desoc. Tiempo de que termino su ultimo trabajo/changa
 'PP07H', # Oc. ppal. Inc. serv. dom. descuento jubilatorio
 'PP07G4', # Oc. ppal. Inc. serv. dom. obra social
 'PP07I', # Oc. ppal. Inc. serv. dom. Aporta jub por sí mismo 
 'PP07G1', # Oc. ppal. Inc. serv. dom. vacaciones pagas
 'CH07'] #Est civil

#en miles de USD
# res_byDPTO = res_DPTO.groupby(['DPTO', 'NOMDPTO', 'NOMPROV'])[variables].mean()
res_byDPTO = res_DPTO.groupby(['RADIO_REF_ID'])[variables].mean()


s = np.round(res_byDPTO, 2).sort_values(by = 'PP07K')#.head() 
s.style.bar(color='#d65f5f')
#  'CAT_OCUP', #CAT_INAC
#  'CAT_INAC', #CAT_INAC

#  'CH08', obra social/salud. nums altos como para mean
# 'CH16', # Donde vivia hace 5. Desconfiable


In [ ]:
# from sklearn.neighbors import NearestNeighbors
# neigh = NearestNeighbors(n_neighbors=1)
# neigh.fit(train[x_cols], train.sample(frac = 1)[y_cols])
# i = 60
# print(neigh.kneighbors([test.iloc[i].values], return_distance=True))
# train.iloc[7022][x_cols] - test.iloc[i]

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
for col in result.columns:
    print('\n')
    print(col)
    df_ = result.loc[result.P03 > 2]
    print(df_[col].value_counts().sort_index())